In [3]:
# This section is for model
import tensorflow as tf

# This section is for algebraic analysis
import numpy as np

# This secion is for visualization
import matplotlib.pyplot as plt
# visualization not crossing the axis border
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')


In [7]:
# help(warnings.filterwarnings)

Help on function filterwarnings in module warnings:

filterwarnings(action, message='', category=<class 'Warning'>, module='', lineno=0, append=False)
    Insert an entry into the list of warnings filters (at the front).
    
    'action' -- one of "error", "ignore", "always", "default", "module",
                or "once"
    'message' -- a regex that the warning message must match
    'category' -- a class that the warning must be a subclass of
    'module' -- a regex that the module name must match
    'lineno' -- an integer line number, 0 matches all warnings
    'append' -- if true, append to the list of filters



types of importing random number:
1. Random: import random as rnd
2. Numpy: import numpy.random as rnd
3. Tensorflow: import tensorflow.random as rnd

<h2> Creating Neural Network <\h2>

In [9]:
# features, observation , dictionary{key:value} (need to know)
# creating parent class
class NeuralNetwork:     # creating constructor
    def __init__(self, layers):      # defining initializer
        self.layers = layers        # creating layers
        self.L = len(layers)       # length of  layers/ no. of info passes in each layer
        
        self.number_features = layers[0]  # taking total no. of features in layers 0
        self.number_class = layers[1]     # taking total no. of classes in layer 1 , we can also use negative index value
        
        
        # to store the weight & bias of each layer we defined dictionary
        self.w = {}  # creating weight by default dictionary
        self.b = {}  # creating bias by default dictionary
        
        self.dw = {}  # creating derivative of weights (derivative = input*weights)
        self.db = {}  # creating derivative of bias 
        
        self.setup()
        
    def setup(self):
        for i in range(1, self.L):      # creating a loop for taking weights & bias for the total no. of layers
            
            self.w[i] = tf.Variable(tf.random.normal(shape = (self.layers[i],self.layers[i-1])))    # taking weight variables randomly and normalize them as dictionary(i is key, i-1 is value of the key)
            self.b[i] =  tf.Variable(tf.random.normal(shape = (self.layers[i],1)))  # bias in initial layer is 1
            
        
        

<h3> Forward Propagation </h3>

In [10]:
#creating child class from parent class
class NeuralNetwork(NeuralNetwork):     #inheritance
    def forwardPass(self, A):           # A is an array
        A = tf.convert_to_tensor(A,dtype = float32)  # converting array value into tensor
        
        for i in range(1, self.L):
            Z= tf.matmul(A, tf.transpose(self.w[i]) + tf.transpose(self.b[i])) #z value formula
            
            if i!= self.L-1:
                A = tf.nn.relu(Z)  # if layer value doesn't match with index value then update array value by activation function relu
            
            else:
                A = Z
       
        return A

<h3> Loss calculation <\h3>

In [11]:
# backpropagation
#creating child class from previous child class

class NeuralNetwork(NeuralNetwork):
    def compute_Loss(self,A,Y):     # A is aaray value , Y i testing value
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(A, Y))  # reducing loss value using sofmax cross entropy
    
    def upgrade_parameters(self, lr):   #upgrading weights and bias on the basis of learning rate
        for i in range (1, self.L):
            self.w[i].assing_sub(lr * self.dw[i])  
            self.b[i].assing_sub(lr * self.db[i])  # substituting the weight and bias drom the deivative and lr product
            
            

<h3> Predicting values after training and testing phase <\h3>

In [13]:
#creating child class from previous child class

class NeuralNetwork(NeuralNetwork):
    def predict(self, x):
        A = self.forwardPass(x) 
        return tf.argmax(tf.nn.softmax(A), axis =1)
    
    def info(self):    
        num_params = 0
        for i in range( 1, self.L):
            num_params += self.w[i].shape[0] * self.w[i].shape[1]  # multiply aweights of features with total observations
            num_params += self.b[i].shape[0]
            
        print("Number of Features: {}".format(self.number_features))
        print("Total number of Class is; {}".format(self.number_class))
        
        print("Hidden Layer information is: {}")
        for j in range(1,self.L-1):   # L-1 is for ommiting input layer
            print("Layer: {}, Units {}".format(i,self.layers[j]))
            
            
        print("Total number of parameters: {}".format(num_params))    
            

<h3> Training Start <\h3>

In [14]:
class NeuralNetwork(NeuralNetwork):
    def train(self, x_train, y_train, x_test, y_test, epochs, step_per_epochs, batch_size, lr):
        history = {"value_loss": [],
                  "train_loss": [],
                  "val_acc": []
                  }